# The Battle of Neighborhoods | Capstone project
# Entertainment Stage

# 1. Reusing the work done in week 3

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [4]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,Postal Code,,Borough,,Neighbourhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


In [5]:
df4 = df3[df3.Borough != 'Not assigned']
df4.head()

,,Postal Code,,Borough,,Neighbourhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


In [6]:
df5 = df4.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df5.head()

,Postal Code,Borough,,,,Neighbourhood,
0,M3A,North York,,,,Parkwoods,
1,M4A,North York,,,,Victoria Village,
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",


In [7]:
url = "http://cocl.us/Geospatial_data"
df6 = pd.read_csv(url)
df6.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
latandlong = pd.merge(df5, df6, on='Postal Code')
latandlong.head(12)

,Postal Code,Borough,,,,Neighbourhood,,Latitude,Longitude
0,M3A,North York,,,,Parkwoods,,43.753259,-79.329656
1,M4A,North York,,,,Victoria Village,,43.725882,-79.315572
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",,43.654260,-79.360636
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",,43.718518,-79.464763
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",,43.662301,-79.389494
5,M9A,Etobicoke,,,,"Islington Avenue, Humber Valley Village",,43.667856,-79.532242
6,M1B,Scarborough,,,,"Malvern, Rouge",,43.806686,-79.194353
7,M3B,North York,,,,Don Mills,,43.745906,-79.352188
8,M4B,East York,,,,"Parkview Hill, Woodbine Gardens",,43.706397,-79.309937
9,M5B,Downtown Toronto,,,,"Garden District, Ryerson",,43.657162,-79.378937


In [9]:
centraltoronto=latandlong[latandlong['Borough'].str.contains('Central')]
centraltoronto

,Postal Code,Borough,,,,Neighbourhood,,Latitude,Longitude
61,M4N,Central Toronto,,,,Lawrence Park,,43.728020,-79.388790
62,M5N,Central Toronto,,,,Roselawn,,43.711695,-79.416936
67,M4P,Central Toronto,,,,Davisville North,,43.712751,-79.390197
68,M5P,Central Toronto,,,,"Forest Hill North & West, Forest Hill Road Park",,43.696948,-79.411307
73,M4R,Central Toronto,,,,"North Toronto West, Lawrence Park",,43.715383,-79.405678
74,M5R,Central Toronto,,,,"The Annex, North Midtown, Yorkville",,43.672710,-79.405678
79,M4S,Central Toronto,,,,Davisville,,43.704324,-79.388790
83,M4T,Central Toronto,,,,"Moore Park, Summerhill East",,43.689574,-79.383160
86,M4V,Central Toronto,,,,"Summerhill West, Rathnelly, South Hill, Forest...",,43.686412,-79.400049


In [10]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... done

# All requested packages already installed.



In [13]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

centraltoronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(centraltoronto['Latitude'], centraltoronto['Longitude'], 
                                           centraltoronto['Borough'], centraltoronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(centraltoronto_map)  
    
centraltoronto_map

# 2. Connecting to foursquer API

##  2.1 Importing the necessaries libraries

In [14]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


##  2.2 Using the credentials

In [16]:
CLIENT_ID = 'G0SV44ZM2YPPPULOYTQCKWPZWKPYKAL3NIQLEJ0L5KRISYZT' # your Foursquare ID
CLIENT_SECRET = '1Q5AILV3DASWPDNEAYRP2QN451FAHH0OSEMLOGTDP0ZEDWAQ' # your Foursquare Secret
ACCESS_TOKEN = 'Q4SLJYLJ1TYWY4T1MMH1VY3D3NAKDP2EBAHJ3TUOEPRZG5T1' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G0SV44ZM2YPPPULOYTQCKWPZWKPYKAL3NIQLEJ0L5KRISYZT
CLIENT_SECRET:1Q5AILV3DASWPDNEAYRP2QN451FAHH0OSEMLOGTDP0ZEDWAQ


### 2.2.1 Defining Eglinton Station as the central point for analysis of the central region

In [20]:
address = '2190 Yonge Street Toronto, Canada' #Eglinton Station

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7061229 -79.39853


##  2.3 Searching for interisting data in foursquare near the Eglinton Station

### 2.3.1 Stores near (10 km)

In [74]:
search_query = 'Store'
radius = 10000
print(search_query + ' .... OK!')

Store .... OK!


In [75]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G0SV44ZM2YPPPULOYTQCKWPZWKPYKAL3NIQLEJ0L5KRISYZT&client_secret=1Q5AILV3DASWPDNEAYRP2QN451FAHH0OSEMLOGTDP0ZEDWAQ&ll=43.7061229,-79.39853&oauth_token=Q4SLJYLJ1TYWY4T1MMH1VY3D3NAKDP2EBAHJ3TUOEPRZG5T1&v=20180604&query=Store&radius=10000&limit=30'

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600b6bbd0860021feb6cf2ba'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4cdb1b04c409b60cffc3d21a',
    'name': 'Internet Kitchen Store',
    'location': {'address': '158 Eglinton Ave W',
     'lat': 43.705734,
     'lng': -79.402993,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.705734,
       'lng': -79.402993}],
     'distance': 361,
     'postalCode': 'M4R 1A7',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['158 Eglinton Ave W',
      'Toronto ON M4R 1A7',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1f8941735',
      'name': 'Furniture / Home Store',
      'pluralName': 'Furniture / Home Stores',
      'shortName': 'Furniture / Home',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
       'suffix': '.png'},
      'primary': True}],
    'referralId'

In [77]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head(10)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4cdb1b04c409b60cffc3d21a,Internet Kitchen Store,"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'F...",v-1611361213,False,158 Eglinton Ave W,43.705734,-79.402993,"[{'label': 'display', 'lat': 43.705734, 'lng':...",361,M4R 1A7,CA,Toronto,ON,Canada,"[158 Eglinton Ave W, Toronto ON M4R 1A7, Canada]",NaN,NaN
1,4c0587458f8fa59358fef10d,The UPS Store,"[{'id': '52f2ab2ebcbc57f1066b8b1f', 'name': 'S...",v-1611361213,False,34 Eglinton Ave W,43.706425,-79.399645,"[{'label': 'display', 'lat': 43.70642476961861...",95,M4R 2H6,CA,Toronto,ON,Canada,"[34 Eglinton Ave W, Toronto ON M4R 2H6, Canada]",NaN,NaN
2,4eefc91e93ad3fada0669385,Mystery Store A,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1611361213,False,NaN,43.706884,-79.398612,"[{'label': 'display', 'lat': 43.70688413850809...",84,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
3,4aeb45dff964a5206ec021e3,The Beer Store,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",v-1611361213,False,609 Roehampton Ave,43.711825,-79.379605,"[{'label': 'display', 'lat': 43.71182475500141...",1649,M4P 1S7,CA,Toronto,ON,Canada,[609 Roehampton Ave (btwn Bayview & Bruce Park...,btwn Bayview & Bruce Park,NaN
4,4e777fe3fa76059701e9418e,3rd Floor Convenience Store,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",v-1611361213,False,2180 Yonge St.,43.705068,-79.398579,"[{'label': 'display', 'lat': 43.70506800967273...",117,NaN,CA,Toronto,ON,Canada,"[2180 Yonge St. (at Eglinton Ave. W.), Toronto...",at Eglinton Ave. W.,NaN
5,4c8fff31590ab1f736ace37d,Mobile Store,[],v-1611361213,False,NaN,43.704488,-79.398944,"[{'label': 'display', 'lat': 43.704488, 'lng':...",185,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
6,4c35ee67a0ced13a7d881a6e,The UPS Store,"[{'id': '52f2ab2ebcbc57f1066b8b1f', 'name': 'S...",v-1611361213,False,3219 Yonge St,43.728270,-79.402966,"[{'label': 'display', 'lat': 43.72826989655437...",2491,M4N 2L3,CA,Toronto,ON,Canada,"[3219 Yonge St (Ranleigh), Toronto ON M4N 2L3,...",Ranleigh,NaN
7,4c993287eaa5a143389dc1e4,Convenience Store,"[{'id': '4bf58dd8d48988d117951735', 'name': 'C...",v-1611361213,False,200 Eglinton East,43.708077,-79.391321,"[{'label': 'display', 'lat': 43.708077, 'lng':...",619,NaN,CA,Toronto,ON,Canada,"[200 Eglinton East, Toronto ON, Canada]",NaN,NaN
8,4ed02765cc219860f72aab4d,The Brick Mattress Store,"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'F...",v-1611361213,False,NaN,43.704865,-79.407319,"[{'label': 'display', 'lat': 43.70486523741324...",721,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
9,4fab155ee4b08190cbfb846c,Happy Lucky Convenience Store,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",v-1611361213,False,Yonge,43.700405,-79.396494,"[{'label': 'display', 'lat': 43.70040535413459...",657,NaN,CA,NaN,NaN,Canada,"[Yonge (Belize), Canada]",Belize,NaN


#### 2.3.1.1 Filtering and ploting the data

In [78]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Internet Kitchen Store,Furniture / Home Store,158 Eglinton Ave W,43.705734,-79.402993,"[{'label': 'display', 'lat': 43.705734, 'lng':...",361,M4R 1A7,CA,Toronto,ON,Canada,"[158 Eglinton Ave W, Toronto ON M4R 1A7, Canada]",NaN,4cdb1b04c409b60cffc3d21a
1,The UPS Store,Shipping Store,34 Eglinton Ave W,43.706425,-79.399645,"[{'label': 'display', 'lat': 43.70642476961861...",95,M4R 2H6,CA,Toronto,ON,Canada,"[34 Eglinton Ave W, Toronto ON M4R 2H6, Canada]",NaN,4c0587458f8fa59358fef10d
2,Mystery Store A,Miscellaneous Shop,NaN,43.706884,-79.398612,"[{'label': 'display', 'lat': 43.70688413850809...",84,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4eefc91e93ad3fada0669385
3,The Beer Store,Beer Store,609 Roehampton Ave,43.711825,-79.379605,"[{'label': 'display', 'lat': 43.71182475500141...",1649,M4P 1S7,CA,Toronto,ON,Canada,[609 Roehampton Ave (btwn Bayview & Bruce Park...,btwn Bayview & Bruce Park,4aeb45dff964a5206ec021e3
4,3rd Floor Convenience Store,Convenience Store,2180 Yonge St.,43.705068,-79.398579,"[{'label': 'display', 'lat': 43.70506800967273...",117,NaN,CA,Toronto,ON,Canada,"[2180 Yonge St. (at Eglinton Ave. W.), Toronto...",at Eglinton Ave. W.,4e777fe3fa76059701e9418e
5,Mobile Store,None,NaN,43.704488,-79.398944,"[{'label': 'display', 'lat': 43.704488, 'lng':...",185,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4c8fff31590ab1f736ace37d
6,The UPS Store,Shipping Store,3219 Yonge St,43.728270,-79.402966,"[{'label': 'display', 'lat': 43.72826989655437...",2491,M4N 2L3,CA,Toronto,ON,Canada,"[3219 Yonge St (Ranleigh), Toronto ON M4N 2L3,...",Ranleigh,4c35ee67a0ced13a7d881a6e
7,Convenience Store,Candy Store,200 Eglinton East,43.708077,-79.391321,"[{'label': 'display', 'lat': 43.708077, 'lng':...",619,NaN,CA,Toronto,ON,Canada,"[200 Eglinton East, Toronto ON, Canada]",NaN,4c993287eaa5a143389dc1e4
8,The Brick Mattress Store,Furniture / Home Store,NaN,43.704865,-79.407319,"[{'label': 'display', 'lat': 43.70486523741324...",721,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4ed02765cc219860f72aab4d
9,Happy Lucky Convenience Store,Convenience Store,Yonge,43.700405,-79.396494,"[{'label': 'display', 'lat': 43.70040535413459...",657,NaN,CA,NaN,NaN,Canada,"[Yonge (Belize), Canada]",Belize,4fab155ee4b08190cbfb846c


In [79]:
dataframe_filtered.name

0                                Internet Kitchen Store
1                                         The UPS Store
2                                       Mystery Store A
3                                        The Beer Store
4                           3rd Floor Convenience Store
5                                          Mobile Store
6                                         The UPS Store
7                                     Convenience Store
8                              The Brick Mattress Store
9                         Happy Lucky Convenience Store
10                                        The UPS Store
11                  Pack and Store Self Storage Toronto
12                               The Beer Store Express
13                                  Drake General Store
14                                    Paper Plane Store
15                      EuropeanGlam down jackets store
16                                    Paper Plane Store
17                                Degrees Kitche

In [61]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Eglinton Station
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Eglinton Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the stores as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### 2.3.2 Restaurants near

In [80]:
search_query = 'Restaurant'
radius = 10000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [81]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=G0SV44ZM2YPPPULOYTQCKWPZWKPYKAL3NIQLEJ0L5KRISYZT&client_secret=1Q5AILV3DASWPDNEAYRP2QN451FAHH0OSEMLOGTDP0ZEDWAQ&ll=43.7061229,-79.39853&oauth_token=Q4SLJYLJ1TYWY4T1MMH1VY3D3NAKDP2EBAHJ3TUOEPRZG5T1&v=20180604&query=Restaurant&radius=10000&limit=30'

In [82]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600b6bd55428e13b76b99005'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4c48e1e36594be9af3245324',
    'name': 'Bikkuri Japanese Restaurant',
    'location': {'address': '1835 Yonge St. #101',
     'crossStreet': 'at Balliol St',
     'lat': 43.69664436996195,
     'lng': -79.39635256109655,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.69664436996195,
       'lng': -79.39635256109655}],
     'distance': 1069,
     'postalCode': 'M4S 1X8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1835 Yonge St. #101 (at Balliol St)',
      'Toronto ON M4S 1X8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d111941735',
      'name': 'Japanese Restaurant',
      'pluralName': 'Japanese Restaurants',
      'shortName': 'Japanese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/ja

In [83]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head(10)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4c48e1e36594be9af3245324,Bikkuri Japanese Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1611361237,False,1835 Yonge St. #101,at Balliol St,43.696644,-79.396353,"[{'label': 'display', 'lat': 43.69664436996195...",1069,M4S 1X8,CA,Toronto,ON,Canada,"[1835 Yonge St. #101 (at Balliol St), Toronto ...",NaN,NaN
1,57289757498e70e0a146f53b,Hannah's Kitchen Restaurant & Catering,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1611361237,False,2177 Yonge St,NaN,43.704753,-79.397946,"[{'label': 'display', 'lat': 43.7047532, 'lng'...",159,M4S 2A9,CA,Toronto,ON,Canada,"[2177 Yonge St, Toronto ON M4S 2A9, Canada]",NaN,NaN
2,4d3b6d6fcc48224b4dea464f,Odessa Restaurant,[],v-1611361237,False,NaN,Hilda and Steeles,43.704145,-79.398331,"[{'label': 'display', 'lat': 43.70414519050178...",220,NaN,CA,Thornhill,ON,Canada,"[Thornhill ON, Canada]",NaN,NaN
3,4ae24e96f964a5200e8d21e3,Coquine Restaurant,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",v-1611361237,False,2075 Yonge St.,at Manor Rd.,43.702623,-79.397429,"[{'label': 'display', 'lat': 43.70262346819862...",399,M4S 2A4,CA,Toronto,ON,Canada,"[2075 Yonge St. (at Manor Rd.), Toronto ON M4S...",329002193,NaN
4,504bcf32e4b0ef19b0e2ecf8,Mt Everest Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1611361237,False,804 Eglinton East,Eglinton and Sutherland,43.713266,-79.366520,"[{'label': 'display', 'lat': 43.713265579296, ...",2695,M4G 2L1,CA,East York,ON,Canada,"[804 Eglinton East (Eglinton and Sutherland), ...",NaN,NaN
5,591984c1b3d8e245316549a3,Maybes Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1611361237,False,188 Eglinton Ave E,NaN,43.708292,-79.392818,"[{'label': 'display', 'lat': 43.7082915052078,...",519,M4P 2X7,CA,Toronto,ON,Canada,"[188 Eglinton Ave E, Toronto ON M4P 2X7, Canada]",NaN,NaN
6,4ad4c05cf964a520dbf520e3,Centro Restaurant & Lounge,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1611361237,False,2472 Yonge St.,NaN,43.711421,-79.399603,"[{'label': 'display', 'lat': 43.71142127575626...",596,M4P 2H5,CA,Toronto,ON,Canada,"[2472 Yonge St., Toronto ON M4P 2H5, Canada]",NaN,NaN
7,4b097010f964a5204b1723e3,Mars Uptown Restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1611361237,False,2363 Yonge St,at Broadway Ave,43.709141,-79.398706,"[{'label': 'display', 'lat': 43.70914074608268...",336,M4P 2C8,CA,Toronto,ON,Canada,"[2363 Yonge St (at Broadway Ave), Toronto ON M...",NaN,NaN
8,4ca609b0f47ea14346845c21,Ho Su Korean & Japanese Restaurant,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1611361237,False,NaN,NaN,43.710002,-79.393856,"[{'label': 'display', 'lat': 43.71000214374543...",572,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
9,4aeb56caf964a52023c121e3,Alizé Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1611361237,False,2459 Yonge St,Yonge and Eglinton,43.711252,-79.399326,"[{'label': 'display', 'lat': 43.711252, 'lng':...",574,M4P 2H6,CA,Toronto,ON,Canada,"[2459 Yonge St (Yonge and Eglinton), Toronto O...",NaN,NaN


#### 2.3.2.1 Filtering and ploting the data

In [84]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Bikkuri Japanese Restaurant,Japanese Restaurant,1835 Yonge St. #101,at Balliol St,43.696644,-79.396353,"[{'label': 'display', 'lat': 43.69664436996195...",1069,M4S 1X8,CA,Toronto,ON,Canada,"[1835 Yonge St. #101 (at Balliol St), Toronto ...",NaN,4c48e1e36594be9af3245324
1,Hannah's Kitchen Restaurant & Catering,Restaurant,2177 Yonge St,NaN,43.704753,-79.397946,"[{'label': 'display', 'lat': 43.7047532, 'lng'...",159,M4S 2A9,CA,Toronto,ON,Canada,"[2177 Yonge St, Toronto ON M4S 2A9, Canada]",NaN,57289757498e70e0a146f53b
2,Odessa Restaurant,None,NaN,Hilda and Steeles,43.704145,-79.398331,"[{'label': 'display', 'lat': 43.70414519050178...",220,NaN,CA,Thornhill,ON,Canada,"[Thornhill ON, Canada]",NaN,4d3b6d6fcc48224b4dea464f
3,Coquine Restaurant,French Restaurant,2075 Yonge St.,at Manor Rd.,43.702623,-79.397429,"[{'label': 'display', 'lat': 43.70262346819862...",399,M4S 2A4,CA,Toronto,ON,Canada,"[2075 Yonge St. (at Manor Rd.), Toronto ON M4S...",NaN,4ae24e96f964a5200e8d21e3
4,Mt Everest Restaurant,Indian Restaurant,804 Eglinton East,Eglinton and Sutherland,43.713266,-79.366520,"[{'label': 'display', 'lat': 43.713265579296, ...",2695,M4G 2L1,CA,East York,ON,Canada,"[804 Eglinton East (Eglinton and Sutherland), ...",NaN,504bcf32e4b0ef19b0e2ecf8
5,Maybes Restaurant,Restaurant,188 Eglinton Ave E,NaN,43.708292,-79.392818,"[{'label': 'display', 'lat': 43.7082915052078,...",519,M4P 2X7,CA,Toronto,ON,Canada,"[188 Eglinton Ave E, Toronto ON M4P 2X7, Canada]",NaN,591984c1b3d8e245316549a3
6,Centro Restaurant & Lounge,Restaurant,2472 Yonge St.,NaN,43.711421,-79.399603,"[{'label': 'display', 'lat': 43.71142127575626...",596,M4P 2H5,CA,Toronto,ON,Canada,"[2472 Yonge St., Toronto ON M4P 2H5, Canada]",NaN,4ad4c05cf964a520dbf520e3
7,Mars Uptown Restaurant,Diner,2363 Yonge St,at Broadway Ave,43.709141,-79.398706,"[{'label': 'display', 'lat': 43.70914074608268...",336,M4P 2C8,CA,Toronto,ON,Canada,"[2363 Yonge St (at Broadway Ave), Toronto ON M...",NaN,4b097010f964a5204b1723e3
8,Ho Su Korean & Japanese Restaurant,Asian Restaurant,NaN,NaN,43.710002,-79.393856,"[{'label': 'display', 'lat': 43.71000214374543...",572,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4ca609b0f47ea14346845c21
9,Alizé Restaurant,Italian Restaurant,2459 Yonge St,Yonge and Eglinton,43.711252,-79.399326,"[{'label': 'display', 'lat': 43.711252, 'lng':...",574,M4P 2H6,CA,Toronto,ON,Canada,"[2459 Yonge St (Yonge and Eglinton), Toronto O...",NaN,4aeb56caf964a52023c121e3


In [85]:
dataframe_filtered.name

0                        Bikkuri Japanese Restaurant
1             Hannah's Kitchen Restaurant & Catering
2                                  Odessa Restaurant
3                                 Coquine Restaurant
4                              Mt Everest Restaurant
5                                  Maybes Restaurant
6                         Centro Restaurant & Lounge
7                             Mars Uptown Restaurant
8                 Ho Su Korean & Japanese Restaurant
9                                   Alizé Restaurant
10                              Good Bite Restaurant
11       The Seven44 Restaurant {e_venue}amp; Lounge
12                              Sorn Thai Restaurant
13                     808 Restaurant - Best western
14         Yitz's Delicatessen Restaurant & Catering
15                                  Bacsi Restaurant
16                                   LOLO restaurant
17                            South China Restaurant
18               Samovar Barmalay Russian Rest

In [86]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Eglinton Station
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Eglinton Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as purple circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map